In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

###ㅅㅂ 근데 잘 안되노
IG_USER_ID = "여기에_ig_user_id"
ACCESS_TOKEN = "여기에_access_token"

# 1) 계정 팔로워 수 가져오기
url_account = f"https://graph.facebook.com/v20.0/{IG_USER_ID}/insights"
params_account = {
    "metric": "follower_count",
    "period": "day",
    "access_token": ACCESS_TOKEN
}
res_account = requests.get(url_account, params=params_account).json()
print(res_account)

# 2) 게시물 목록 가져오기
url_media = f"https://graph.facebook.com/v20.0/{IG_USER_ID}/media"
params_media = {
    "fields": "id,caption,media_type,media_product_type,timestamp",
    "access_token": ACCESS_TOKEN
}
res_media = requests.get(url_media, params=params_media).json()
print(res_media)


hellworld
